In [ ]:
import optuna
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
import os
warnings.filterwarnings("ignore")

configs = [
    dict(
        symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='sucessive',
        train_start=datetime(2020,2,1), train_end=datetime(2025,2,1),
        test_start=datetime(2022,2,1),  test_end=datetime(2023,2,1),
        search_type='clusters', search_subtype='lgmm', search_filter='mapie',
        label_method='random', n_models=1, debug=False,
    ),
]

# Actualizar n_jobs en cada configuración
for cfg in configs:
    cfg["tag"] = f"{cfg['symbol']}_{cfg['timeframe']}_{cfg['direction']}_{cfg['search_type']}_{cfg.get('search_subtype', '')}_{cfg.get('search_filter', '')}_{cfg['label_method']}".replace("__", "_").strip("_")

DB_FILE = f"optuna_dbs/{cfg['tag']}.db"
DB_PATH = f"sqlite:///{DB_FILE}"
STUDY_NAME = f"{cfg['tag']}"

study = None
if not os.path.exists(DB_FILE):
    study = None
else:
    try:
        study = optuna.load_study(study_name=STUDY_NAME, storage=DB_PATH)
    except Exception:
        study = None

n_trials = 500
if study:
    n_trials = len(study.trials) + n_trials

for cfg in configs:
    cfg['n_trials'] = n_trials

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD_H1_buy_random_clusters_lgmm] modelo 0 trial 9/508 score=0.439428 avg=4.53s mem=473.25MB 
[XAUUSD_H1_buy_random_clusters_lgmm] modelo 0 trial 10/508 score=0.439428 avg=8.48s mem=642.22MB 
[XAUUSD_H1_buy_random_clusters_lgmm] modelo 0 trial 11/508 score=0.439428 avg=9.26s mem=654.09MB 
[XAUUSD_H1_buy_random_clusters_lgmm] modelo 0 trial 12/508 score=0.439428 avg=13.67s mem=742.93MB 
[XAUUSD_H1_buy_random_clusters_lgmm] modelo 0 trial 13/508 score=0.439428 avg=15.70s mem=743.50MB 
[XAUUSD_H1_buy_random_clusters_lgmm] modelo 0 trial 14/508 score=0.439428 avg=17.08s mem=743.86MB 
[XAUUSD_H1_buy_random_clusters_lgmm] modelo 0 trial 15/508 score=0.439428 avg=19.78s mem=749.58MB 
[XAUUSD_H1_buy_random_clusters_lgmm] modelo 0 trial 16/508 score=0.439428 avg=21.35s mem=737.40MB 
[XAUUSD_H1_buy_random_clusters_lgmm] modelo 0 trial 17/508 score=0.493986 avg=21.08s mem=735.07MB 
[XAUUSD_H1_buy_random_clusters_lgmm] modelo 0 trial 18/508 score=0.493986 avg=24.03s mem=751.54MB 
[XAUUSD_H1_buy